In [33]:
import torch
# from d2l import torch as d2l
from torch import nn

def corr2d(X,K):   #X是输入，，K是核矩阵
    #计算二维互相关运算 
    h,w = K.shape   #K 的行数和列数
    Y = torch.zeros((X.shape[0] - h+1,X.shape[1]-w+1))   #Y--输出的高度和宽度
    for i in range(Y.shape[0]):
        for j in range (Y.shape[1]):
            Y[i,j] = (X[i:i +h,j:j +w] * K).sum()   #X的第i行到第i+h行，第j列到第j+w列   再和K做点积，再求和
    return Y
#实现多通道的互相关运算
def corr2d_multi_in(X,K):
    return sum(corr2d(x,k) for x,k in zip(X,K))

In [10]:
#验证互相关运算的输出
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [26]:
#计算多个通道的输出的互相关函数
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2])

In [29]:
B = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
B = torch.stack((B, B + 1, B + 2), 0)
B.shape

torch.Size([3, 2, 2, 2])

In [30]:
corr2d_multi_in_out(X,B)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [39]:
#1X1的卷积等于全连接

def corr2d_multi_in_out_1X1(X,B):
    c_i,h,w = X.shape
    c_o = B.shape[0]
    X = X.reshape((c_i,h * w))
    B = B.reshape((c_o,c_i))
    Y = torch.matmul(B,X)
    return Y.reshape((c_o,h,w))

X = torch.normal(0, 1, (3, 3, 3))
B = torch.normal(0, 1, (2, 3, 1, 1))  #输入是3，输出是2,1X1

Y1 = corr2d_multi_in_out_1X1(X, B)
Y2 = corr2d_multi_in_out(X, B)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6


In [40]:
def comp_conv2d(conv2d,X):
    X = X.reshape((1,1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])